<h1>serial Predictions</h1>

Dieses Notebook möchte zeigen, wie *detectron2* seine Voraussagen für die Klassen von Bildern ermittelt und wie man diese seriell erzeugen, speichern und graphisch auswerten kann.

Bezug genommen wird hier auf das Programm "serialDetection.py". Es folgen zunächst grundlegende Importe.

In [3]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
import cv2, json

Im folgenden Schritt geben wir den Ort der zu erkennenden Bilder an. Ziel des Ganzen ist auch, dass später der gesamte Datensatz so eingelesen werden kann.

In [4]:
path = "/home/julius/PowerFolders/Masterarbeit/Bilder/1_Datensaetze/first_annotation_dataset/"
    
image_list = sorted([picture for picture in os.listdir(path) if picture.endswith(".jpg")])

print(image_list)


['acht1_036.jpg', 'acht1_067.jpg', 'acht1_100.jpg', 'acht1_158.jpg', 'acht2_024.jpg', 'acht2_085.jpg', 'acht2_140.jpg', 'acht2_153.jpg', 'acht2_242.jpg', 'acht5_030.jpg']


Die nun folgenden Zeilen bestimmen die Konfiguration des Modells und sollten sich in allen Versuchsanordnungen ähneln.
Entscheidend ist hierbei das initialisieren der Gewichte. Dieses Projekt hat nciht den Anspruch ein Modell von Grundauf neu zu trainieren. Vielmehr wird auf ein bereits bestehendes Modell aufgebaut.  
Grundlage für die in dieser Arbeit beschriebenen Modelle wird das Mask R-CNN für COCO Instance Segmentation "R50-FPN", welches in *detectron2* enthalten ist, sein. Eine genauere Dokumentation können sie [hier](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md) finden. Wichtig für uns ist, dass es bei einer geringen Trainingszeit relativ gute Ergebnisse liefert.

In [5]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)

In der letzten Zeile des obigen Codeblocks wurde mit Hilfe der Konfiguration der Predictor gebaut. Dieser ist das Kernelement dieses Abschnitts und erzeugt die Ergebnisse, die wir dann analysieren wollen.  
Im nächsten Abschnitt wird nun über jedes Bild im Zielordner Iteriert und ein output erzeugt.

In [6]:
outputs = {}
for image in image_list:
    img = cv2.imread(path + image)
    outputs[image] = predictor(img)


In diesen Outputs befinden sich nun für jedes Bild die einzelnen Instanzen - Die Klassen Indices und die dazugehörigen Wahrscheinlichkeiten.  
Bevor wir nun mit dem Speichern der Werte fortfahren machen wir noch einen kleinen Exkurs für die Konfiguration. In der Konfiguration werden auch die Klassennamen vorgegeben. Eine Liste der Klassennamen kann man sich mit folgendem Befehl anzeigen lassen:

In [7]:
print(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes)
class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes


['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


speichern wir nun in anderen Variablen.  
Hierbei wollen wir das generelle Format des Outputs ein wenig anpassen. Wir behalten die Grundstruktur mit den Bildern als Hauptschlüsseln bei. Den einzelnen Schlüsseln geben wir dann jeweils ein Dictionary mit den Schlüsseln zu jeder "Instance". Diese besitzen wiederum Dictionarys mit den jeweiligen Schlüssel-Wert-Paaren für "Class" "Class_name" und "Score".

In [8]:
converted_outputs = {}


In [9]:
for image in image_list:
    converted_outputs[image] = {}
    for element in range(len(outputs[image]["instances"].scores)):
        converted_outputs[image]["instance{}".format(element)] = {}
        converted_outputs[image]["instance{}".format(element)]["Class"] = outputs[image]["instances"].pred_classes[element].item()
        converted_outputs[image]["instance{}".format(element)]["Class_name"] = class_names[converted_outputs[image]["instance{}".format(element)]["Class"]]
        converted_outputs[image]["instance{}".format(element)]["Score"] = outputs[image]["instances"].scores[element].item()

Wir haben nun Angaben für jedes Bild, was der Algorithmus erkannt haben will und wie sicher er sich dabei war.  
Wenn man nun davon ausgeht, dass man diese Ergebnisse von allen Bildern bekommen wird, so lassen sich daraus erste Ableitungen herführen. So kann man ermitteln, wie viele Instanzen von Klassen er über einem bestimmten Schwellwert erkannt hat; Wie hoch die durchschnittliche Genauigkeit für eine Klasse mit und ohne Schwellwert ist. Was man jedoch damit überhaupt nicht schließen kann ist, ob er einen Gegenstand fälschlicherweise als diesen bezeichnet und ob er Gegenstände aus dieser Kategorie nicht erkannt hat. Ebenso lassen sich keine Aussagen über die Bounding Boxes oder die Segmentierung treffen - auch wenn man die Daten darüber gespeichert hätte.  
Hierfür bedarf es eines Datensatzes, der angibt was wo zu erkennen ist und auf dessen Grundlage man Annotationen bewerten kann. Dieser liegt aber für den gesamten Datensatz nicht vor und würde sich im Rahmen dieser Arbeit nicht in einer ausreichenden Größe anfertigen lassen (Ausreichend meint hier wenigstens >10%, was 1.500 Bildern entsprechen würde). Es bedarf also einem Behelf.  
*detectron2* liefert neben dem allgemeinen Predictor auch einen allgemeinen Evaluator. Dieser benötigt jedoch wie oben beschrieben einen Datensatz mit dem er Verifizieren kann. Der Evaluator wird dazu dienen einen doppelten Boden für die Analysen einzuziehen. Dieser ist von nöten, da durch das Vorgehen und die oben beschriebene Problematik ein bruch in der Arbeitsweise vorhanden ist. Möchte man das untrainierte Modell am Datensatz testen, kann man nicht mit einer sogenanten Ground Truth oder eben dem Datensatz für die Validierung die Ergebnisse verifizieren. Man ist also auf die wenigen Daten, die dieses Notebook liefet angewiesen. In der nächsten Phase ist nun angedacht das Modell auf einen anderen COCO-Datensatz zu trainieren. Konkret geht es um den [COCO-Text-Datensatz](https://bgshih.github.io/cocotext/), welcher schon Annotationen für Bild und Text-Erkennung in einem liefert. Dieses Model könnte man dann sowohl mit diesen Predictions rudimentär auswerten und hätte gegebenenfalls Vergleichswerte zur ersten Phase, als auch mit dem allgemeinen Evaluator neue, präzisere Erkenntnisse gewinnen. In einer dritten Phase könnte dann auf einem kleinen Datensatz das zweite Modell weiter auf die Plakatsammlung angepasst werden. In dieser Phase wären wieder beide Evualationsmethoden anwendbar und könnten miteinander verglichen werden. In einer weiteren Versuchsanordnung könnte man ebenfalls die letzten beiden Phasen vertauschen und ermitteln ob es qualitative Unterschiede in den beiden Ergebnissen gibt. Ebenso könnte jeweils nur eine der beiden Phasen verwendet werden.

Um dieses Notebook abzuschließen wurde im folgenden das Dictionary von Werten befreit, die unter einer Wahrscheinlichkeit von 50% lagen. Die 50% ergeben sich aus der Verfahrensweise von *detectron2*. Hier werden in der Visualisierung der Annotationsergebnisse auch nur die Elemente berücksichtigt, die eine Wahrscheinlichkeit von über 50% besitzen.

In [16]:
to_be_deleted = {}
for image in converted_outputs:
    to_be_deleted[image] = []
    for instance in converted_outputs[image]:
        if converted_outputs[image][instance]["Score"] < 0.50:
            to_be_deleted[image].append(instance)
for image in to_be_deleted:
    for element in to_be_deleted[image]:
        converted_outputs[image].pop(element)

Aus diesen so umgewandelten Ergebnissen sollen nun die statistischen Werte erhoben werden. Die folgenden Zeilen Code sammeln hierfür die Klassennamen und ihre Häufigkeit in einer absoluten Zahl, die durchschnittliche Wahrscheinlichkeit für jede Klasse, sowie die Anzahl der Plakate auf denen Nichts erkannt worden ist.

In [19]:
detected_names = {} 
detected_scores = {}
zero_detections = 0

for image in converted_outputs:
    if len(converted_outputs[image]) == 0:
        zero_detections += 1
    for instance in converted_outputs[image]:
        if converted_outputs[image][instance]["Class_name"] not in detected_names:
            detected_names[converted_outputs[image][instance]["Class_name"]] = 1
            detected_scores[converted_outputs[image][instance]["Class_name"]] = converted_outputs[image][instance]["Score"]
        else:
            detected_names[converted_outputs[image][instance]["Class_name"]] += 1
            detected_scores[converted_outputs[image][instance]["Class_name"]] += converted_outputs[image][instance]["Score"]

for element in detected_scores:
    detected_scores[element] = detected_scores[element]/detected_names[element]

print(detected_names)
print(detected_scores)
print(zero_detections)

{'horse': 1, 'person': 12, 'bottle': 1, 'clock': 3, 'vase': 1, 'teddy bear': 1, 'kite': 2}
{'horse': 0.9984381794929504, 'person': 0.8588288178046545, 'bottle': 0.9010133147239685, 'clock': 0.6946642597516378, 'vase': 0.5634121894836426, 'teddy bear': 0.587200939655304, 'kite': 0.622450053691864}
1


In [45]:
%matplotlib inline

import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, constrained_layout=False)
axs[0] = plt.bar(range(len(detected_names)), list(detected_names.values()), align="center")
axs[0] = plt.xticks(range(len(detected_names)), list(detected_names.keys()))
axs[1] = plt.bar(range(len(detected_scores)), list(detected_scores.values()), align="center")
axs[1] = plt.xticks(range(len(detected_scores)), list(detected_scores.keys()))
plt.show()

TypeError: float() argument must be a string or a number, not 'Rectangle'

Die beiden folgenden Blöcke dienen der Sicherung der Daten

In [17]:
with open(path + "converted_outputs.json", "w") as output_file:
    json.dump(converted_outputs, output_file, indent=4)

In [55]:
with open(path + "outputs.txt", "w") as output_file:
    for line in outputs:
        output_file.write("{}:{}\n".format(line, outputs[line]))